# P5 - Convolutioneel neuraal netwerk

In [34]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential


## Data preparation

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print("x_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("x_test shape", X_test.shape)
print("y_test shape", y_test.shape)

170508288/170498071 [==============================] - 13s 0us/step
x_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
x_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


Voor one hot encoding uit over de target values, en laten we ook de dataset normaliseren

In [3]:
# One hot encodign over de targets heen
Y_train = utils.to_categorical(y_train, 10)
Y_test = utils.to_categorical(y_test, 10)

# normaliseren van de data
X_train = utils.normalize(X_train)
X_test = utils.normalize(X_test)

## Modelling

Voor het berekenen van de output dimension kunnen we de volgende formule gebruiken.

    Output height = (Input height + padding height top + padding height bottom - kernel height) / (stride height) + 1 
    
    Output width = (Output width + padding width right + padding width left - kernel width) / (stride width) + 1

Aangezien het voor dit dataset altijd uitkomt naar een vierkant hoeven we maar 1 te gebruiken om de size te weten.

In [56]:
cnn = Sequential()

# Layer 1 (CN)
# mijn hoop is door de size voor nu nog het zelfe de te houden dat we meer detail kunnen opvangen.
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", input_shape=(32,32,3)))
cnn.add(BatchNormalization(axis=-1))
cnn.add(Activation("relu"))


# Layer 2 (CN)
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(2, 2)))
cnn.add(BatchNormalization(axis=-1))
cnn.add(Activation("relu"))

# Layer 3 (CN)
cnn.add(Conv2D(filters=16, kernel_size=(3, 3)))
cnn.add(BatchNormalization(axis=-1))
cnn.add(Activation("relu"))
cnn.add(MaxPool2D(pool_size=(2, 2)))
cnn.add(Flatten())

# Layer 4 (FC)
cnn.add(Dense(512))
cnn.add(BatchNormalization(axis=-1))
cnn.add(Activation("relu"))

# Layer 5 (FC)
cnn.add(Dense(200))
cnn.add(Activation("relu"))

# Layer 6 (FC)
cnn.add(Dense(10))
cnn.add(Activation("softmax"))


### Summary
Hieronder kunnen we dan de summary zien van het gehele netwerk

In [57]:
cnn.summary(
)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_35 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_22 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 activation_67 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_36 (Conv2D)          (None, 15, 15, 32)        9248      
                                                                 
 batch_normalization_23 (Bat  (None, 15, 15, 32)       128       
 chNormalization)                                                
                                                                 
 activation_68 (Activation)  (None, 15, 15, 32)      

### Compilen
Ook hier moeten we het model gaan compilen met een loss function en optimizer.

In [58]:
cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fitten
Hierna kunnen we het model gaan fitten. De fit functie kent een aantal belangrijke parameters:

In [59]:
cnn.fit(X_train, Y_train, batch_size=64, epochs=50, verbose=1)

Epoch 1/50
782/782 [==============================] - 18s 22ms/step - loss: 1.5623 - accuracy: 0.4418
Epoch 2/50
782/782 [==============================] - 17s 22ms/step - loss: 1.2542 - accuracy: 0.5581
Epoch 3/50
782/782 [==============================] - 17s 22ms/step - loss: 1.0900 - accuracy: 0.6142
Epoch 4/50
782/782 [==============================] - 17s 22ms/step - loss: 0.9617 - accuracy: 0.6614
Epoch 5/50
782/782 [==============================] - 17s 22ms/step - loss: 0.8402 - accuracy: 0.7037
Epoch 6/50
782/782 [==============================] - 17s 22ms/step - loss: 0.7273 - accuracy: 0.7448
Epoch 7/50
782/782 [==============================] - 17s 22ms/step - loss: 0.6123 - accuracy: 0.7864
Epoch 8/50
782/782 [==============================] - 17s 22ms/step - loss: 0.5194 - accuracy: 0.8157
Epoch 9/50
782/782 [==============================] - 18s 22ms/step - loss: 0.4301 - accuracy: 0.8503
Epoch 10/50
782/782 [==============================] - 18s 23ms/step - loss: 0.358

## Stap 4. Evalueren

Ook hier kunnen we kijken hoe goed de resultaten zijn.

In [31]:
loss, accuracy = cnn.evaluate(X_test, Y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

313/313 [==============================] - 1s 2ms/step - loss: 2.9148 - accuracy: 0.4486
Test loss: 2.9148
Test accuracy: 0.4486
